# Maximum Likelihood Estimation

Our goal in this article is to derive a cost function for which to optimize various parameters that will be used in a team sports model.

We first define an ELO scheme as follows (see https://en.wikipedia.org/wiki/Elo_rating_system for more details about ELO):

Assume that a competition has $N$ teams, and consists of $M$ matches. We assign each team an initial ELO rating for the first match (regardless of whether they are competing). For each match $m\in M$, Each team $n \in N$ has an ELO rating denoted by $ELO_n^m \in\mathbb{R}$.

Let $j,k \in N$ be two teams competing in the $m$th match. Then the probability of team $j$ winning the match is given by

$$p_j^m = \frac{1}{1+e^\frac{-(ELO_j^m-ELO_k^m)}{400}}$$

After the $m$th match, we update the ELO ratings as follows

\begin{align}
    ELO_n^{m+1} &= ELO_n^m + k \times (result_n^m - p_n^m) &\text{for all teams $n \in N$ that competed in match $m$}
\end{align}

where $result_n^m$ is $1$ if team $n$ wins the $m$th match and $0$ if they lose.

Using mathematical induction (see article "Get Your Own Variable" for a similar proof), we can show that for any $m \in M$ and $n \in N$, $ELO_n^m$ and $p_n^m$ are both merely functions of:

1. The results of all past matches team $n$ has played.
2. The initial ELO ratings of all teams.
3. The parameter $k$

Let $E$ denote the set of all initial ELO ratings, and $R_n^m$ denote the set of all results for team $n\in N$ before match $m\in M$. That is, $E = \{ELO_n^1: n\in N\}$ and $R_n^m = \{result_n^i: i\leq (m-1) \text{ for } m\in M\}$. We can now think of $p_m^n$ as a function that takes as arguments the parameters $R_n^m$, $E$, $k$ for some $n\in N$ and $m \in M$, and returns the probability that team $n$ wins match $m$. That is, $p_m^n: (R_n^m, E, k) \mapsto (0,1)$.

The most common approach to making a model to predict future results, is to train our parameters on past data and see which setting makes the best predictions. One method of doing this is finding what's called the Maximum Likelihood Estimation. Here is the setup for our case:

We make the assumption that the $result_n^m$s are independent Bernoulli random variables each with it's own unknown parameter $p_n^m$. Note that these $p_n^m$s differ because they are parameterized by $R_n^m$ which will be a different set for each match. This means the probability mass function for each $result_n^m$ is given by

$$f(result_n^m\,|\,p_n^m) = {p_n^m}^{result_n^m}(1-p_n^m)^{1-result_n^m}$$.

For brevity, let $results$ denote the set of all results in the competition, and $p$ denote the set of all parameters. That is, $results = \{result_n^m:n\in N\text{ and }m\in M\}$ and $p = \{p_n^m:n\in N\text{ and }m\in M\}$. The Maximum Likelihood Estimation seeks the set $p$ that maximizes the likelihood function:

$$\hat{p}_{ML} := arg\,\underset{p}{max}\, f(results\,|\,p)$$

Due to the monotonicity of the log function, we have that 

$$ arg\,\underset{p}{max}\, f(results\,|\,p) = arg\,\underset{p}{max}\, \log(f(results\,|\,p))$$

Since our results are independent, it follows that

\begin{align}
\log(f(results\,|\,p)) &= log(\prod_{i=1}^N \prod_{j=1}^M {p_i^j}^{result_i^j}(1-p_i^j)^{1-result_i^j}) \\
&= \sum_{i=1}^N \sum_{j=1}^M log({p_i^j}^{result_i^j}(1-p_i^j)^{1-result_i^j}) \\
&= \sum_{i=1}^N \sum_{j=1}^M result_i^j \log(p_i^j)+(1-result_i^j)\log(1-p_i^j). \\
\end{align}

and thus

\begin{align}
\hat{p}_{ML} &= arg\,\underset{p}{max}\, \log(f(results\,|\,p)) \\
&= arg\,\underset{p}{max}\, \sum_{i=1}^N \sum_{j=1}^M result_i^j \log(p_i^j)+(1-result_i^j)\log(1-p_i^j) \\
\end{align}

Since $p$ is paramterised by $R_n^m$, $E$ and $k$, we can make a further simplification. $R_n^m$ is a dependent variable which we cannot vary, and we can set each team's initial ELO rating to the same value. Now, finding the $p$ that maximizes the maximum likelihood equation is the equivalent to finding the $k$ that maximizes it. So we are left with

$$ \hat{p}_{ML} = arg\,\underset{k}{max}\, \sum_{i=1}^N \sum_{j=1}^M result_i^j \log(p_i^j)+(1-result_i^j)\log(1-p_i^j) $$

We now have a cost function that we can apply to historical data to find the optimal $k$.